In [88]:
import os
import requests
from bs4 import BeautifulSoup
import textwrap
import pandas as pd
import lxml  
import time
from IPython.display import clear_output
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import re
from tqdm import tqdm
from functools import reduce

In [89]:
data = pd.read_csv('data.csv')
data

,average_rating,price,reviews_num,max_cpu_frequency,cpu_architecture,web_camera,weight,ethernet_adapter,pixel_response_time,microphone,...,micro_sdhc,micro_sdxc,glossy_display,screen_width_pixels,screen_height_pixels,hdd_increased_speed,best_wi_fi_standard,li_ion_battery_type,ultrabook,touch_numeric_keypad
0,1.0,17999.0,1.0,2.2,GoldMont,True,0.935,False,30.0,True,...,False,False,0.0,1280,800,False,4,True,False,False
1,4.2,24999.0,17.0,2.8,Goldmont Plus,True,1.050,False,NaN,True,...,True,True,0.0,1366,768,False,5,True,False,False
2,4.7,24999.0,3.0,2.8,Goldmont Plus,True,1.050,False,NaN,True,...,True,True,0.0,1366,768,False,5,False,False,False
3,4.6,24999.0,9.0,2.8,Goldmont Plus,True,1.050,False,NaN,True,...,True,True,0.0,1366,768,False,5,True,False,False
4,3.8,28999.0,9.0,2.6,Excavator,True,2.000,False,NaN,True,...,True,True,0.0,1366,768,False,5,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,NaN,313999.0,NaN,4.5,Comet Lake,True,2.100,True,NaN,True,...,False,False,0.0,3840,2160,False,6,True,False,False
725,3.0,349999.0,2.0,4.5,Comet Lake,True,2.100,True,NaN,True,...,False,False,0.0,3840,2160,False,5,True,False,False
726,NaN,379999.0,NaN,5.0,Comet Lake,True,2.540,True,NaN,True,...,False,False,0.0,3840,2160,False,6,True,False,False
727,NaN,380999.0,NaN,4.5,Comet Lake,True,2.100,True,NaN,True,...,False,False,0.0,3840,2160,False,5,True,False,False


In [90]:
data.columns.values

array(['average_rating', 'price', 'reviews_num', 'max_cpu_frequency',
       'cpu_architecture', 'web_camera', 'weight', 'ethernet_adapter',
       'pixel_response_time', 'microphone', 'warranty_duration', 'depth',
       'screen_diagonal', 'battery_capacity', 'gaming_laptop',
       'kensington_lock', 'cpu_cores_num', 'l2_cache', 'l3_cache',
       'cpu_family', 'screen_refresh_rate', 'threads_num',
       'max_possible_ram', 'integrated_graphics_card_model',
       'dedicated_graphics_card_model', 'cpu_model',
       'screen_resolution_name', 'hdd_capacity', 'ssd_capacity',
       'graphics_ram', 'intel_optane_drive_capacity', 'os',
       'optical_disc_drive', 'ppi', 'usb_power_delivery',
       'keyboard_backlight', 'usb_2_0_ports', 'usb_3_x_ports',
       'usb_type_c_ports', 'estimated_battery_life', 'gpu_manufacturer',
       'cpu_manufacturer', 'ram', 'M_2_connector', 'sound_system',
       'touch_screen', 'fingerprint_scanner', 'cpu_lithography',
       'graphics_ram_type', 'ra

In [91]:
data.dtypes[data.dtypes == object]

cpu_architecture                  object
cpu_family                        object
integrated_graphics_card_model    object
dedicated_graphics_card_model     object
cpu_model                         object
screen_resolution_name            object
os                                object
gpu_manufacturer                  object
cpu_manufacturer                  object
sound_system                      object
graphics_ram_type                 object
ram_type                          object
display_type                      object
color                             object
color_gamut                       object
additional_info                   object
adaptive_sync_technology          object
brand_name                        object
model_name                        object
dtype: object

In [92]:
def col_info(col_name = None):
    print(f'col_name = {col_name}')
    print(f'col_dtype = {data[col_name].dtype}\n')
    print('unique_values_counts:\n')
    print(data[col_name].value_counts(dropna = False))

## CPU Info

In [93]:
data.cpu_manufacturer + ' ' + data.cpu_architecture + ' ' + data.cpu_family + ' ' + data.cpu_model  

0           Intel GoldMont Celeron N3450
1      Intel Goldmont Plus Celeron N4020
2      Intel Goldmont Plus Celeron N4020
3      Intel Goldmont Plus Celeron N4020
4                  AMD Excavator A4 9125
                     ...                
724       Intel Comet Lake Core i7 9750H
725       Intel Comet Lake Core i7 9750H
726      Intel Comet Lake Core i7 10750H
727       Intel Comet Lake Core i7 9750H
728     Intel Coffee Lake Core i9 9980HK
Length: 729, dtype: object

In [94]:
col_info('cpu_manufacturer')

col_name = cpu_manufacturer
col_dtype = object

unique_values_counts:

Intel    528
AMD      199
Apple      2
Name: cpu_manufacturer, dtype: int64


In [95]:
col_info('cpu_architecture')

col_name = cpu_architecture
col_dtype = object

unique_values_counts:

Comet Lake       236
Zen 2            124
Ice Lake         123
Coffee Lake       63
Willow Cove       51
Zen               34
Zen+              31
Goldmont Plus     24
Whiskey Lake      14
Excavator         10
Tiger Lake         8
NaN                5
Amber Lake-Y       2
GoldMont           2
Broadwell          1
Kaby Lake          1
Name: cpu_architecture, dtype: int64


In [96]:
col_info('cpu_family')

col_name = cpu_family
col_dtype = object

unique_values_counts:

Core i7    225
Core i5    199
Ryzen 5     76
Ryzen 7     56
Core i3     46
Pentium     40
Ryzen 3     37
Athlon      14
Core i9     10
Celeron      8
Ryzen 9      6
A4           5
A9           3
A6           2
M1           2
Name: cpu_family, dtype: int64


In [97]:
col_info('cpu_model')

col_name = cpu_model
col_dtype = object

unique_values_counts:

10750H       68
10300H       51
1035G1       41
1005G1       35
10210U       33
             ..
N3350         1
8250U         1
N3450         1
300U          1
Gold 7505     1
Name: cpu_model, Length: 62, dtype: int64


In [98]:
# CPU architecture, family and manufacturer will be One-Hot-Encoded

# For every CPU model we download correspoding benchmark value

In [99]:
class Driver(object):
    def __init__(self, chromedriver_path, headless = True):
        options = Options()
        options.headless = headless
#         options.add_argument('--headless')
#         options.add_argument('window-size=1920x1080')
        
        self.driver = webdriver.Chrome(chromedriver_path, options = options)

    def getLink(self, link_text, waiting_time = 1, niters = 5, raise_error = True):
        for _ in range(niters):
            try:
                return self.driver.find_element_by_partial_link_text(link_text)
            except:
                sleep(waiting_time)
        
        if raise_error:
            raise(Exception("Driver: Failed to get link"))
        
    def getHTML(self, url, waiting_time = 2):
        self.driver.get(url)
        
        sleep(waiting_time)

        content = self.driver.page_source

        return BeautifulSoup(content, 'html.parser')
    
    def processRequest(self, url, soup_function, waiting_time = 1, niters = 1, raise_error = True, load_url = True):
        if load_url:
            self.driver.get(url)
            sleep(waiting_time)
        
        for _ in range(niters):
            try:
                soup = BeautifulSoup(self.driver.page_source, 'html.parser')
                return soup_function(soup)
            except:
                sleep(waiting_time)
        
        if raise_error:
            raise(Exception("Driver: Failed to process request"))
        
chromedriver_path = r'C:\Users\Yuriy\Downloads\chromedriver_win32_4\chromedriver.exe'
driver = Driver(chromedriver_path, headless = False)

In [100]:
def getCPUscores():
    soup = driver.getHTML('https://www.cpubenchmark.net/cpu_list.php', waiting_time = 3)
    cpu_table = soup.find('table', {'id': 'cputable'})
    cpu_scores = pd.read_html(str(cpu_table))[0]
    cpu_scores = cpu_scores.rename(columns = {'CPU Name': 'name', 'CPU Mark(higher is better)': 'score'})
    cpu_scores = cpu_scores.drop(['Rank(lower is better)', 'CPU Value(higher is better)', 'Price(USD)'], axis = 1)
    return cpu_scores

cpu_scores = getCPUscores()

# Apple M1 similar to Intel Core i7-8700B 
# (found on the Internet)
cpu_scores = cpu_scores.append({'name': 'M1', 'score': 12548}, ignore_index=True)

# Intel Pentium Gold 7505 similar to Intel Core i3-10110U
# (found on the Internet)
cpu_scores = cpu_scores.append({'name': 'Intel Pentium Gold 7505', 'score': 4092}, ignore_index=True)

cpu_scores

,name,score
0,AMD 3015e,2678
1,AMD 3020e,2650
2,AMD A4 Micro-6400T APU,1004
3,AMD A4 PRO-3340B,1706
4,AMD A4 PRO-7300B APU,1448
...,...,...
3008,VIA QuadCore U4650 @ 1.0+ GHz,666
3009,ZHAOXIN KaiXian KX-U6780A@2.7GHz,3744
3010,ZHAOXIN KaiXian ZX-C+ C4700@2.0GHz,1547
3011,M1,12548


In [101]:
def foo(name, keyword):
    for word in name.split():
        if word.find(keyword) != -1:
            return word.split('-')[-1]
    return np.nan

def bar(name, keyword):
    for word in name.split():
        if word.find(keyword) != -1:
            return word.split('-')[-1]
    return np.nan

def find_name(idx):
    manufacturer = data.cpu_manufacturer.loc[idx]
    architecture = data.cpu_architecture.loc[idx] if pd.notnull(data.cpu_architecture.loc[idx]) else 'NONE'
    family = data.cpu_family.loc[idx]
    model = data.cpu_model.loc[idx]
    
    keyword = model.split()[-1]
    
    scores = cpu_scores[cpu_scores.name.map(lambda name: name.find(keyword) != -1)].copy()
    
    if len(scores) == 1:
        return scores.score.values[0]
    
    if len(scores) == 0:
        print(idx, 'NOT FOUND')
        return -1
    
    scores['foo'] = scores.name.map(lambda name: foo(name, keyword))
    scores['match'] = scores.foo == keyword
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]
    
    scores['foo'] = scores.name.map(lambda name: foo(name, manufacturer))
    scores['match'] = scores.foo == manufacturer
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]
    
    scores['foo'] = scores.name.map(lambda name: " ")
    scores['match'] = scores.name.map(lambda name: name.lower().find(" ".join([family.lower(), model.lower()])) != -1)
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]
    
    print(idx, 'NOT FOUND')
    return -2

In [102]:
# cpu models were missing on the dns-shop site
# found on the internet

data['cpu_model'][data.cpu_model == '0'] =\
            ['5350U', '5350U', '8257U', '1038NG7', '1038NG7', '1038NG7', '9750H', '9750H', '9880H', '9880H']

C:\Users\Yuriy\.conda\envs\New\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [103]:
data['cpu_scores'] = [find_name(idx) for idx in data.index.values]

In [104]:
data.drop('cpu_model', axis = 1, inplace = True, errors = 'ignore')

## GPU Info

In [105]:
data.integrated_graphics_card_model    

0       Intel HD Graphics 500
1               Intel UHD 600
2               Intel UHD 600
3               Intel UHD 600
4                   Radeon R3
                ...          
724    Intel UHD Graphics 630
725    Intel UHD Graphics 630
726    Intel UHD Graphics 630
727    Intel UHD Graphics 630
728    Intel UHD Graphics 630
Name: integrated_graphics_card_model, Length: 729, dtype: object

In [106]:
data.gpu_manufacturer + ' ' + data.dedicated_graphics_card_model    

0                               Intel нет
1                               Intel нет
2                               Intel нет
3                               Intel нет
4                                 AMD нет
                      ...                
724          nVidia GeForce RTX 2080 MaxQ
725                nVidia Quadro RTX 5000
726    nVidia GeForce RTX 2080 Super MaxQ
727                nVidia Quadro RTX 5000
728               nVidia GeForce RTX 2080
Length: 729, dtype: object

In [107]:
col_info('gpu_manufacturer')

col_name = gpu_manufacturer
col_dtype = object

unique_values_counts:

nVidia    329
Intel     251
AMD       147
Apple       2
Name: gpu_manufacturer, dtype: int64


In [108]:
col_info('integrated_graphics_card_model')

col_name = integrated_graphics_card_model
col_dtype = object

unique_values_counts:

Intel UHD Graphics 630    208
Intel UHD                 111
Intel UHD Graphics 620     59
Intel Iris Xe              54
Radeon Vega 6              48
Radeon Vega 7              47
Intel Iris Plus            47
Radeon Vega 8              34
Radeon Vega 3              26
Intel UHD 605              17
Radeon Vega 5              17
Radeon Vega 2               8
Radeon Vega 10              6
Intel UHD Graphics          6
Intel UHD 600               6
Radeon R3                   5
Intel UHD Graphics 610      5
отключена                   5
Radeon R5                   3
Radeon Graphics             3
Intel UHD Graphics 605      3
Intel HD 6000               2
M1 8-core                   2
Intel UHD 615               2
Radeon R4                   2
Intel HD Graphics 500       1
Intel Iris Plus 645         1
Intel HD 500                1
Name: integrated_graphics_card_model, dtype: int64


In [109]:
col_info('dedicated_graphics_card_model')

col_name = dedicated_graphics_card_model
col_dtype = object

unique_values_counts:

нет                            371
GeForce GTX 1650                54
GeForce RTX 2060                48
GeForce GTX 1650 Ti             44
GeForce GTX 1660 Ti             35
GeForce MX350                   25
GeForce RTX 2070 Super          14
GeForce MX250                   14
GeForce GTX 1660 Ti MaxQ        14
GeForce RTX 2070                12
GeForce MX330                   11
GeForce RTX 2080 Super MaxQ      8
GeForce GTX 1650 MaxQ            7
GeForce RTX 2060 MaxQ            7
GeForce RTX 2070 MaxQ            6
Radeon 540X                      6
GeForce MX110                    6
Quadro RTX3000                   5
GeForce RTX 2070 Super MaxQ      5
GeForce GTX 1650 Ti MaxQ         4
GeForce RTX 2080                 4
Radeon RX 5500M                  4
GeForce RTX 2080 Super           3
GeForce MX450                    3
GeForce RTX 2080 MaxQ            3
Radeon Pro 5500M                 2
Radeon

In [110]:
# manufacturer -> OHE
# integrated and dedicated -> benchmark values

In [111]:
def getGPUscores():
    soup = driver.getHTML('https://www.videocardbenchmark.net/gpu_list.php', waiting_time = 3)
    gpu_table = soup.find('table', {'id': 'cputable'})
    gpu_table = pd.read_html(str(gpu_table))[0]
    gpu_table = gpu_table.rename(columns = {'Videocard Name': 'name', 'Passmark G3D Mark (higher is better)': 'score'})
    gpu_table = gpu_table.drop(['Rank (lower is better)', 'Videocard Value (higher is better)', 'Price (USD)'], axis = 1)
    return gpu_table

gpu_scores = getGPUscores()

# Radeon R5 similar to Radeon HD 7470M
# (found on the Internet)
gpu_scores = gpu_scores.append({'name': 'Radeon R5', 'score': 407}, ignore_index=True)

# Radeon Vega 5 similar to AMD Radeon RX 550
# (found on the Internet)
gpu_scores = gpu_scores.append({'name': 'Radeon Vega 5', 'score': 2790}, ignore_index=True)
 
# Radeon Vega 7 similar to Radeon HD 7850
# (found on the Internet)
gpu_scores = gpu_scores.append({'name': 'Radeon Vega 7', 'score': 3730}, ignore_index=True)

# M1 8-core similar to Radeon Pro WX 8200
# (found on the Internet)
gpu_scores = gpu_scores.append({'name': 'M1 8-core', 'score': 14189}, ignore_index=True)
    
gpu_scores

,name,score
0,3DP Edition,826
1,9xx Soldiers sans frontiers Sigma 2,21
2,15FF,8229
3,64MB DDR GeForce3 Ti 200,5
4,64MB GeForce2 MX with TV Out,2
...,...,...
2175,ZX Chrome 645/640 GPU,147
2176,Radeon R5,407
2177,Radeon Vega 5,2790
2178,Radeon Vega 7,3730


In [112]:
data.dedicated_graphics_card_model.unique()

array(['нет', 'GeForce MX110', 'Radeon R530', 'GeForce MX330',
       'Radeon 540X', 'Radeon 620', 'Radeon 625', 'Radeon RX640',
       'GeForce MX250', 'GeForce MX350', 'GeForce MX230', 'GeForce MX150',
       'GeForce MX130', 'GeForce GTX 1650', 'GeForce GTX 1650 Ti',
       'Radeon RX 5500M', 'GeForce GTX 1050', 'GeForce RTX 2060',
       'GeForce GTX 1660 Ti', 'GeForce GTX 1660 Ti MaxQ', 'GeForce MX450',
       'GeForce GTX 1650 MaxQ', 'GeForce RTX 2060 MaxQ',
       'GeForce RTX 2070', 'GeForce RTX 2070 MaxQ',
       'GeForce GTX 1650 Ti MaxQ', 'GeForce RTX 2070 Super',
       'Quadro T1000', 'GeForce RTX 2080 MaxQ',
       'GeForce RTX 2070 Super MaxQ', 'GeForce RTX 2080',
       'GeForce RTX 2080 Super MaxQ', 'Radeon Pro 5300M',
       'Quadro RTX3000', 'GeForce RTX 2080 Super', 'Radeon Pro 5500M',
       'Quadro RTX 5000'], dtype=object)

In [113]:
def getNumKeyword(model):
    for word in model.split():
        for letter in word:
            if letter.isdigit():
                return word

def find_dedicated_graphics_card_name(idx):
    model = data.dedicated_graphics_card_model.loc[idx]
    
    if model == 'нет':
        return 0
    
    if model == 'Radeon R530':
        model = 'Radeon 530'
     
    if model == 'Radeon RX640':
        model = 'Radeon RX 640'
        
    if model == 'Quadro RTX3000':
        model = 'Quadro RTX 3000'
        
    model = model.replace('MaxQ', 'with Max-Q Design')
    
    keyword = getNumKeyword(model)

    scores = gpu_scores[gpu_scores.name.map(lambda name: name.find(keyword) != -1)].copy()
    
    if len(scores) == 1:
        return scores.score.values[0]
    
    if len(scores) == 0:
        print(idx, model, 'NOT FOUND')
        return -1
    
    scores['match'] = scores.name.map(lambda name: name.find(model) != -1)
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]
    
    scores['match'] = scores.name.map(lambda name: name == model)
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]

    print(idx, 'NOT FOUND')
    return -2

In [114]:
data['dedicated_gpu_scores'] = [float(find_dedicated_graphics_card_name(idx)) for idx in data.index.values]

In [115]:
data.integrated_graphics_card_model.unique()

array(['Intel HD Graphics 500', 'Intel UHD 600', 'Radeon R3', 'Radeon R5',
       'Radeon Vega 3', 'Intel UHD', 'Radeon Vega 2', 'Intel UHD 605',
       'Intel UHD Graphics 610', 'Radeon R4', 'Intel HD 500',
       'Intel UHD Graphics', 'Intel UHD Graphics 605', 'Radeon Vega 5',
       'Intel UHD Graphics 620', 'Radeon Vega 8', 'Radeon Vega 6',
       'Intel Iris Xe', 'Radeon Vega 10', 'Radeon Vega 7',
       'Radeon Graphics', 'Intel UHD Graphics 630', 'Intel Iris Plus',
       'Intel HD 6000', 'Intel UHD 615', 'отключена',
       'Intel Iris Plus 645', 'M1 8-core'], dtype=object)

In [116]:
def getNumKeyword(model):
    for word in model.split():
        for letter in word:
            if letter.isdigit():
                return word
    return ''

def find_integrated_graphics_card_name(idx):
    model = data.integrated_graphics_card_model.loc[idx]
    
    if model == 'отключена':
        return np.nan
        
    model = model.replace('UHD Graphics', 'HD')
    model = model.replace('HD Graphics', 'HD')
    
    if model == 'Intel HD 605':
        model = 'Intel UHD 605'
    
    if model == 'Intel HD 620':
        model = 'Intel HD Graphics 620'

    if model == 'Intel HD' or model == 'Radeon Graphics':
        return np.nan
    
    keyword = getNumKeyword(model)

    scores = gpu_scores[gpu_scores.name.map(lambda name: name.find(keyword) != -1)].copy()
    
    if len(scores) == 1:
        return scores.score.values[0]
    
    if len(scores) == 0:
        print(idx, model, 'NOT FOUND')
        return -1
    
    scores['match'] = scores.name.map(lambda name: name.find(model) != -1)
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]
    
    scores['match'] = scores.name.map(lambda name: name == model)
    
    if len(scores[scores.match == True]) == 1:
        return scores[scores.match == True].score.values[0]
    
    print(idx, f'keyword = {keyword}, model = {model}')
    print(scores[scores.name.map(lambda name: name.find(model.split()[0]) != -1)])
    
    return

    print(idx, 'NOT FOUND')
    return -2

In [117]:
for idx in data.index.values:
    print(find_integrated_graphics_card_name(idx))

307
333
333
333
307
307
333
307
333
407
897
407
1920
333
460
897
366
307
1920
1920
1920
1920
667
366
366
366
667
307
339
366
460
460
1920
1920
460
460
366
366
1920
460
366
1920
307
1920
366
1920
897
897
897
1920
nan
339
366
897
667
897
1920
1920
897
366
1920
460
1920
667
1920
2790
407
366
460
1920
897
1920
1920
1920
897
1920
897
nan
1920
366
366
366
366
1920
1920
1920
897
1920
897
366
897
897
1920
1920
1920
1920
897
nan
667
897
366
1920
897
897
1920
897
897
897
2790
920
1578
1920
1920
2790
1920
1578
897
1920
2790
2790
1920
366
1920
2790
1578
1578
2790
2790
1920
1920
1920
366
1578
1255
897
1920
1920
897
1920
2790
1920
897
1578
2790
2790
2790
1255
920
1578
1578
1578
1578
1920
1920
1920
2790
1578
1578
1920
1920
1255
1920
1920
1920
1255
1255
1578
1578
1920
1920
920
920
2790
1255
1920
1920
1578
1920
1920
1920
1255
1578
1578
920
920
1920
1920
1920
2918
1920
1255
1255
2790
1255
1255
1920
1920
920
1920
1920
1920
1920
1255
1920
1524
1920
1255
1255
920
1524
1920
1920
2918
1920
1578
1578
1255
125

In [118]:
data['integrated_gpu_scores'] = [float(find_integrated_graphics_card_name(idx)) for idx in data.index.values]

In [119]:
data[['integrated_gpu_scores', 'dedicated_gpu_scores']].max(axis = 1)

0        307.0
1        333.0
2        333.0
3        333.0
4        307.0
        ...   
724    13971.0
725    16706.0
726    13889.0
727    16706.0
728    18578.0
Length: 729, dtype: float64

In [120]:
data.drop('integrated_graphics_card_model', axis = 1, inplace = True, errors = 'ignore')
data.drop('dedicated_graphics_card_model', axis = 1, inplace = True, errors = 'ignore')

## Screen Info

In [121]:
data.screen_resolution_name

0       HD
1       HD
2       HD
3       HD
4       HD
      ... 
724    UHD
725    UHD
726    UHD
727    UHD
728    UHD
Name: screen_resolution_name, Length: 729, dtype: object

In [122]:
data.display_type

0          IPS
1      TN+film
2      TN+film
3      TN+film
4      TN+film
        ...   
724        IPS
725        IPS
726        IPS
727        IPS
728        IPS
Name: display_type, Length: 729, dtype: object

In [123]:
data.adaptive_sync_technology

0      NaN
1      нет
2      нет
3      нет
4      нет
      ... 
724    нет
725    нет
726    нет
727    нет
728    нет
Name: adaptive_sync_technology, Length: 729, dtype: object

In [124]:
data.color_gamut

0      49% sRGB, 45% NTSC, 34% AdobeRGB
1                              45% NTSC
2                              45% NTSC
3                              45% NTSC
4                                   NaN
                     ...               
724                       100% AdobeRGB
725                       100% AdobeRGB
726                       100% AdobeRGB
727                       100% AdobeRGB
728                       100% AdobeRGB
Name: color_gamut, Length: 729, dtype: object

In [125]:
col_info('screen_resolution_name')

col_name = screen_resolution_name
col_dtype = object

unique_values_counts:

FullHD    639
UHD        38
HD+        18
HD         16
WQXGA      10
QHD         5
WQXGA+      1
WXGA+       1
WQHD        1
Name: screen_resolution_name, dtype: int64


In [126]:
col_info('display_type')

col_name = display_type
col_dtype = object

unique_values_counts:

IPS              635
TN+film           77
SVA (TN+film)      9
WVA                4
OLED               4
Name: display_type, dtype: int64


In [127]:
col_info('adaptive_sync_technology')

col_name = adaptive_sync_technology
col_dtype = object

unique_values_counts:

нет              559
NaN               68
NVIDIA G-SYNC     67
Adaptive-Sync     29
AMD FreeSync       6
Name: adaptive_sync_technology, dtype: int64


In [128]:
# above - OHE

In [129]:
col_info('color_gamut')

col_name = color_gamut
col_dtype = object

unique_values_counts:

NaN                                 222
45% NTSC                            181
100% sRGB                           111
72% NTSC                             40
100% AdobeRGB                        29
50% AdobeRGB, 67% sRGB, 45% NTSC     14
54% sRGB                             13
62% sRGB, 45% NTSC                   10
60% NTSC                              7
100% sRGB, 72% NTSC                   7
39% AdobeRGB, 62% sRGB                6
94% sRGB                              5
100% DCI-P3                           5
77% AdobeRGB                          5
36% AdobeRGB, 57% sRGB                4
85% sRGB, 63% AdobeRGB                4
62% sRGB, 39% AdobeRGB                4
92% sRGB, 60% AdobeRGB                4
48% AdobeRGB, 64% sRGB, 45% NTSC      3
48% AdobeRGB, 65% sRGB, 46% NTSC      3
91% sRGB, 59% AdobeRGB                3
67% sRGB, 50% AdobeRGB                3
47% AdobeRGB, 62% sRGB, 45% NTSC      3
100% sRGB, 67%

In [130]:
vals = list(map(lambda s: s.split(','), data.color_gamut.value_counts().index.unique().values))
vals = list(reduce(lambda l1, l2: l1 + l2, vals))
vals = np.array(list(map(lambda val: val.split(), vals)))
np.unique(vals[:, 1])

array(['AdobeRGB', 'DCI-P3', 'NTSC', 'sRGB'], dtype='<U8')

In [131]:
def foo(s, keyword):
    if pd.isnull(s):
        return np.nan
    
    vals = s.split(',')
    for val in vals:
        if val.split()[1] == keyword:
            return int(val.split()[0][:-1])
    return np.nan

In [132]:
# missing values:
# mean
# median 
# moda 
# simple 0 with column-indicator

In [133]:
data['AdobeRGB'] = data.color_gamut.map(lambda s: np.nan if pd.isnull(s) else foo(s, 'AdobeRGB'))

In [134]:
col_info('AdobeRGB')

col_name = AdobeRGB
col_dtype = float64

unique_values_counts:

NaN      614
100.0     29
50.0      21
39.0      10
48.0       9
77.0       5
37.0       5
63.0       5
60.0       5
36.0       5
47.0       5
67.0       3
59.0       3
41.0       3
65.0       2
58.0       2
38.0       1
64.0       1
34.0       1
Name: AdobeRGB, dtype: int64


In [135]:
data['DCI-P3'] = data.color_gamut.map(lambda s: np.nan if pd.isnull(s) else foo(s, 'DCI-P3'))

In [136]:
col_info('DCI-P3')

col_name = DCI-P3
col_dtype = float64

unique_values_counts:

NaN      722
100.0      5
90.0       2
Name: DCI-P3, dtype: int64


In [137]:
data['NTSC'] = data.color_gamut.map(lambda s: np.nan if pd.isnull(s) else foo(s, 'NTSC'))

In [138]:
col_info('NTSC')

col_name = NTSC
col_dtype = float64

unique_values_counts:

NaN     443
45.0    221
72.0     49
60.0      7
46.0      5
70.0      4
Name: NTSC, dtype: int64


In [139]:
data['sRGB'] = data.color_gamut.map(lambda s: np.nan if pd.isnull(s) else foo(s, 'sRGB'))

In [140]:
col_info('sRGB')

col_name = sRGB
col_dtype = float64

unique_values_counts:

NaN      490
100.0    130
62.0      27
67.0      19
54.0      13
58.0       5
94.0       5
85.0       5
65.0       5
57.0       5
64.0       4
92.0       4
91.0       3
59.0       3
56.0       3
99.0       2
95.0       1
80.0       1
98.0       1
60.0       1
89.0       1
49.0       1
Name: sRGB, dtype: int64


In [141]:
data.drop('color_gamut', axis = 1, inplace = True, errors = 'ignore')

AttributeError: 'Series' object has no attribute 'model_name'

## RAM Info

In [142]:
data.ram_type

0      DDR4
1      DDR4
2      DDR4
3      DDR4
4      DDR4
       ... 
724    DDR4
725    DDR4
726    DDR4
727    DDR4
728    DDR4
Name: ram_type, Length: 729, dtype: object

In [143]:
data.graphics_ram_type

0        SMA
1        SMA
2        SMA
3        SMA
4        SMA
       ...  
724    GDDR6
725    GDDR6
726    GDDR6
727    GDDR6
728    GDDR6
Name: graphics_ram_type, Length: 729, dtype: object

In [144]:
col_info('ram_type')

col_name = ram_type
col_dtype = object

unique_values_counts:

DDR4       633
LPDDR4x     58
LPDDR3      25
LPDDR4      12
DDR3L        1
Name: ram_type, dtype: int64


In [145]:
# maybe label encoding if can be ranked
# otherwise just OHE...

In [146]:
col_info('graphics_ram_type')

col_name = graphics_ram_type
col_dtype = object

unique_values_counts:

SMA      367
GDDR6    261
GDDR5     97
NaN        4
Name: graphics_ram_type, dtype: int64


In [147]:
# SMA -> graphics_ram_type == ram_type

## OS Info

In [148]:
data.os

0              Windows 10 Home
1              Windows 10 Home
2              Windows 10 Home
3              Windows 10 Home
4                UNIX-подобная
                ...           
724    Windows 10 Professional
725    Windows 10 Professional
726    Windows 10 Professional
727    Windows 10 Professional
728    Windows 10 Professional
Name: os, Length: 729, dtype: object

In [149]:
col_info('os')

col_name = os
col_dtype = object

unique_values_counts:

Windows 10 Home            427
DOS                        115
UNIX-подобная               90
Windows 10 Professional     74
macOS                       12
Windows 10 S                11
Name: os, dtype: int64


In [150]:
# OHE

## Sound Info  

In [151]:
data.sound_system

0                         стереодинамики
1            ASUS SonicMaster , ICEpower
2            ASUS SonicMaster , ICEpower
3            ASUS SonicMaster , ICEpower
4                       ASUS SonicMaster
                     ...                
724                      Waves MaxxAudio
725                       стереодинамики
726                       стереодинамики
727                      Waves MaxxAudio
728    Waves MaxxAudio , четыре динамика
Name: sound_system, Length: 729, dtype: object

In [152]:
col_info('sound_system')

col_name = sound_system
col_dtype = object

unique_values_counts:

стереодинамики                                                 112
Acer TrueHarmony                                               103
Dolby Atmos                                                     57
Dolby Audio                                                     54
DTS:X Ultra                                                     46
два динамика                                                    41
NaN                                                             32
Smart AMP                                                       30
Nahimic 3                                                       27
ASUS SonicMaster , ICEpower                                     26
Harman kardon                                                   23
Bang & Olufsen с 2 динамиками, HP Audio Boost                   20
HP Audio Boost , Bang & Olufsen с 2 динамиками                  15
Bang & Olufsen с 2 динамиками                                 

In [153]:
# for now just drop

In [154]:
data.drop('sound_system', axis = 1, inplace = True, errors = 'ignore')

## Color

In [155]:
data.color

0       черный
1        синий
2      розовый
3        белый
4       черный
        ...   
724      белый
725      белый
726      белый
727      белый
728     черный
Name: color, Length: 729, dtype: object

In [156]:
col_info('color')

col_name = color
col_dtype = object

unique_values_counts:

черный         336
серый          143
серебристый    134
синий           41
белый           37
красный         13
розовый         11
зеленый         10
золотистый       4
Name: color, dtype: int64


In [157]:
# does not affect price (at least that's what I believe in...)

In [158]:
data.drop('color', axis = 1, inplace = True, errors = 'ignore')

## Other Info

In [159]:
data.additional_info

0            NaN
1      NumberPad
2      NumberPad
3      NumberPad
4            NaN
         ...    
724          NaN
725          NaN
726          NaN
727          NaN
728          NaN
Name: additional_info, Length: 729, dtype: object

In [160]:
data.brand_name

0      Digma
1       ASUS
2       ASUS
3       ASUS
4       ASUS
       ...  
724     Acer
725     Acer
726     Acer
727     Acer
728     Acer
Name: brand_name, Length: 729, dtype: object

In [161]:
data.model_name

0                        EVE 10 C301
1               Laptop E210MA-GJ001T
2               Laptop E210MA-GJ002T
3               Laptop E210MA-GJ003T
4                Laptop K540BA-GQ613
                   ...              
724         ConceptD 7 CN715-71-79YB
725    ConceptD 7 Pro CN715-71P-77A7
726    ConceptD 7 Ezel CC715-71-70X8
727    ConceptD 7 Pro CN715-71P-70XB
728         ConceptD 9 CN917-71-964C
Name: model_name, Length: 729, dtype: object

In [162]:
col_info('additional_info')

col_name = additional_info
col_dtype = object

unique_values_counts:

NaN                                                                                                                             576
NumberPad                                                                                                                        37
шторка веб-камеры                                                                                                                13
датчик Холла                                                                                                                     11
акселерометр                                                                                                                     11
датчик Холла , датчик внешней освещенности                                                                                        8
совмещенный разъем Type-C / DisplayPort, акселерометр                                                                             7
поддер

In [163]:
tmp = list(map(lambda s: s.replace(' ', '').split(','), data['additional_info'].value_counts().index.unique().values))
print(np.unique(list(reduce(lambda l1, l2: l1 + l2, tmp))))

['12.6”Full-HDдополнительныйсенсорныйдисплей'
 '14”4K/UHD(3840x1100)дополнительныйсенсорныйдисплей'
 '4-хзоннаяподсветкаклавитуры' '720pHDвеб-камера' 'Bluetooth4.0'
 'Multi-Touch' 'Nahimic3' 'NumberPad' 'ИК-датчиктемпературы'
 'акселерометр' 'быстраязарядка' 'веб-камера1.0Мпикс'
 'влагозащищеннаяклавиатура' 'встроенныйкриптопроцессорстандартаTPM2.0'
 'гироскоп' 'датчикХолла' 'датчиквнешнейосвещенности'
 'датчикповоротаэкрана' 'датчикприближения'
 'дисплейсосветодиоднойподсветкой'
 'защитноестеклоэкранаCorningGorillaGlass'
 'защитноестеклоэкранаGorillaGlass' 'керамическаяповерхность'
 'крышкаAnimematrixLEDDisplay' 'панельTouchBarсовстроеннымдатчикомTouchID'
 'поддержкастандартаMiracast' 'совмещенныйразъемType-C/DisplayPort'
 'стеклянноепокрытиеCorningGorillaGlassNBT'
 'уголраскрытияэкрана180градусов' 'шторкавеб-камеры' 'электронныйкомпас']


In [164]:
# to be done....

In [165]:
data.drop('additional_info', axis = 1, inplace = True, errors = 'ignore')

In [166]:
col_info('model_name')

col_name = model_name
col_dtype = object

unique_values_counts:

Yoga Slim 7 14IIL05             18
IdeaPad 5 14ARE05                6
Legion 7 15IMH05                 5
IdeaPad 3 17ADA05                5
IdeaPad 5 15ARE05                5
                                ..
Laptop F509JA-BQ675T             1
Aspire 3 A317-52-35E1            1
VivoBook Flip TM420IA-EC085T     1
Nitro 5 AN515-55-56AQ            1
GL75 10SCXR-013RU                1
Name: model_name, Length: 647, dtype: int64


In [167]:
# just to be dropped

In [168]:
data.drop('model_name', axis = 1, inplace = True, errors = 'ignore')

In [169]:
col_info('brand_name')

col_name = brand_name
col_dtype = object

unique_values_counts:

Acer      223
ASUS      177
Lenovo    112
HP        109
MSI        58
Huawei     14
Apple      12
Dell       12
Honor      11
Digma       1
Name: brand_name, dtype: int64


In [170]:
# OHE

In [171]:
# maybe drop Digma

In [172]:
data = data.drop(0)

In [173]:
data.to_csv('processed_data.csv', index = False)